In [62]:
# convert pdf to text file

import fitz  # PyMuPDF
import os

def pdf_to_text_file(pdf_path, output_txt_path):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    
    # Extract all text
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    
    # Save to a text file
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write(full_text)
    
    doc.close()
    print(f"Text saved to: {output_txt_path}")

# Example usage
pdf_path = "/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type/jsr-17-task-002_aiforhealthandhealthcare12122017.pdf"
output_txt_path = "/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type/aiforhealthandhealthcare.txt"

pdf_to_text_file(pdf_path, output_txt_path)


Text saved to: /Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type/aiforhealthandhealthcare.txt


In [3]:
# text pre-processing

import re

def preprocess_pdf_text(text):
    # Step 1: Remove page numbers (assuming they appear on a line by themselves)
    text = re.sub(r'\n\d+\n', '\n', text)

    # Step 2: Remove common headers and footers (heuristic)
    lines = text.split('\n')
    cleaned_lines = []
    for line in lines:
        if len(line.strip()) < 3:  # Skip short lines (e.g., single letters or digits)
            continue
        if re.match(r'^(Page|PAGE)\s*\d+', line.strip()):  # Page indicators
            continue
        cleaned_lines.append(line)
    text = '\n'.join(cleaned_lines)

    # Step 3: Fix hyphenated line breaks (e.g., "inter-\nview" → "interview")
    text = re.sub(r'(\w+)-\n(\w+)', r'\1\2', text)

    # Step 4: Merge lines that are artificially split (end of line not a sentence end)
    # First, normalize line endings
    text = re.sub(r'\r\n?', '\n', text)

    # Then merge lines that are not paragraph breaks
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)  # convert single line breaks to spaces

    # Step 5: Normalize multiple newlines (paragraph breaks)
    text = re.sub(r'\n{2,}', '\n\n', text)

    # Step 6: Normalize whitespace
    text = re.sub(r'[ \t]+', ' ', text)  # collapse tabs and spaces
    text = re.sub(r' +\n', '\n', text)   # trim trailing spaces on lines
    text = text.strip()

    return text


In [4]:
with open("/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type/aiforhealthandhealthcare.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

clean_text = preprocess_pdf_text(raw_text)

with open("/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type/cleaned_aiforhealthandhealthcare.txt", "w", encoding="utf-8") as f:
    f.write(clean_text)


In [40]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize


def fine_grained_chunks(text, max_tokens=300):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_tokens = 0

    def estimate_tokens(sentence):
        return int(len(word_tokenize(sentence)) / 0.75)

    for sent in sentences:
        sent_tokens = estimate_tokens(sent)
        if current_tokens + sent_tokens <= max_tokens:
            current_chunk.append(sent)
            current_tokens += sent_tokens
        else:
            if current_chunk:
                chunks.append(' '.join(current_chunk))
            current_chunk = [sent]
            current_tokens = sent_tokens

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks


In [41]:
with open("/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type/cleaned_aiforhealthandhealthcare.txt", "r", encoding="utf-8") as f:
    text = f.read()

chunks = fine_grained_chunks(text, max_tokens=300)  # ~225 words max

print(f"Total chunks: {len(chunks)}")
print(f"First chunk:\n{chunks[0]}")


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/winnie/Documents/nltk_data/tokenizers/punkt/english.pickle'
**********************************************************************


In [24]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Ensure punkt is downloaded to a custom directory
nltk.download('punkt', download_dir='/Users/winnie/nltk_data')
nltk.data.path.append('/Users/winnie/nltk_data')

def fine_grained_chunks(text, max_tokens=300):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_tokens = 0

    def estimate_tokens(sentence):
        return int(len(word_tokenize(sentence)) / 0.75)

    for sent in sentences:
        sent_tokens = estimate_tokens(sent)
        if current_tokens + sent_tokens <= max_tokens:
            current_chunk.append(sent)
            current_tokens += sent_tokens
        else:
            if current_chunk:
                chunks.append(' '.join(current_chunk))
            current_chunk = [sent]
            current_tokens = sent_tokens

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks


[nltk_data] Downloading package punkt to /Users/winnie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# Force path and download if needed
# nltk.download('punkt', download_dir='/Users/winnie/Documents/nltk_data/tokenizers/punkt')
nltk.data.path.clear()
nltk.data.path.append('/Users/winnie/Documents/nltk_data')


def fine_grained_chunks(text, max_tokens=300):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_tokens = 0

    def estimate_tokens(sentence):
        return int(len(word_tokenize(sentence)) / 0.75)

    for sent in sentences:
        sent_tokens = estimate_tokens(sent)
        if current_tokens + sent_tokens <= max_tokens:
            current_chunk.append(sent)
            current_tokens += sent_tokens
        else:
            if current_chunk:
                chunks.append(' '.join(current_chunk))
            current_chunk = [sent]
            current_tokens = sent_tokens

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Read file and chunk
with open("/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type/cleaned_aiforhealthandhealthcare.txt", "r", encoding="utf-8") as f:
    text = f.read()

chunks = fine_grained_chunks(text, max_tokens=300)
print(f"Total chunks: {len(chunks)}")
print(f"First chunk:\n{chunks[0]}")


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/winnie/Documents/nltk_data'
**********************************************************************


In [44]:
import nltk
nltk.download('punkt', download_dir='/Users/winnie/Documents/nltk_data')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/winnie/Documents/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [49]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.data.path.clear()
nltk.data.path.append('/Users/winnie/Documents/nltk_data')

text = "Hello world. This is a test. Let's fix this error."
print(sent_tokenize(text))


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/winnie/Documents/nltk_data'
**********************************************************************


In [51]:
print(nltk.data.path)


['/Users/winnie/Documents/nltk_data']


In [55]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/winnie/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [56]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

def fine_grained_chunks(text, max_tokens=300):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    current_tokens = 0

    def estimate_tokens(sentence):
        return int(len(word_tokenize(sentence)) / 0.75)

    for sent in sentences:
        sent_tokens = estimate_tokens(sent)
        if current_tokens + sent_tokens <= max_tokens:
            current_chunk.append(sent)
            current_tokens += sent_tokens
        else:
            if current_chunk:
                chunks.append(' '.join(current_chunk))
            current_chunk = [sent]
            current_tokens = sent_tokens

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Read file and chunk
with open("/Users/winnie/Documents/GitHub/MedAI/Analysis&NLP/classify_by_AI_type/cleaned_aiforhealthandhealthcare.txt", "r", encoding="utf-8") as f:
    text = f.read()

chunks = fine_grained_chunks(text, max_tokens=300)
print(f"Total chunks: {len(chunks)}")
print(f"First chunk:\n{chunks[0]}")

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/winnie/Documents/nltk_data'
**********************************************************************
